# Pull Polygons from TerraMatch API for PPC (Simple)

This notebook sets up the process to pull polygon geometries and metadata from the TerraMatch API specifically for PPC projects (includes the hectares by
WWF ecoregion indicator).

Updated to include indicators and simplify preprocessing

In [ ]:
import yaml
import pandas as pd
from tm_api_utils import pull_tm_api_data, patch_tm_api_data
from tqdm import tqdm
import json
import sys
from datetime import datetime
sys.path.append('../src/')
import api_utils as api
import process_tm_api_results as clean

## Set file paths

In [ ]:
## PARAMS
# Naming convention
run_dir = 'ppc'
run_name = 'ppc_batch2'

# Today's date
today = datetime.today().strftime('%Y-%m-%d') # Check computer date before running (if out of sync, run sudo hwclock -s)

In [ ]:
## FILES
# Input Files
# List of approved projects
approved_projects_file = '../projects_all_approved_202502211226.csv'

# PPC 2025 Batch 2 Projects
ppc_batch2_file = '../data/ppc/ppc_2025_batch2_project_list.csv'

# PPC 2025 Potential Tree Count Projects (Eligibility Check #1)
#ppc_tree_count_file = '../data/ppc_tree_count_elig/ppc_2025_potential_tree_count_projects_2025-07-22.csv'

# Output Files
# JSON file to store the results of the TM API pull; read it back in to clean the results (outfile, infile)
tm_api_pull_results_file = f'../data/{run_dir}/tm_api_response_prod_{run_name}_{today}.json'

# CSV file to save the results of the TM API pull
polygon_features_file = f'../data/{run_dir}/tm_api_{run_name}_{today}.csv'

## Set up token & API URL

In [ ]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [ ]:
# TerraMatch API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # Use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

## Create list of projects to pull

In [ ]:
# Read in list of approved projects (2025-02-21)
full = pd.read_csv(approved_projects_file)

# Read in list of projects
batch2_projects = pd.read_csv(ppc_batch2_file)

In [ ]:
# Create lists of projects by Cohort (and split cohort 1 into projects within the TF landscapes and outside of the TF landscapes)
cohort1 = full[full['cohort'] == 'terrafund']
cohort1_landscapes = cohort1[cohort1['country'].isin(['BI', 'CD', 'RW', 'KE', 'GH'])]
cohort1_non_landscapes = cohort1[~cohort1['country'].isin(['BI', 'CD', 'RW', 'KE', 'GH'])]
cohort2 = full[full['cohort'] == 'terrafund-landscapes']

ppc = full[full['cohort'] == 'ppc']

In [ ]:
# Create a list of project ids to query
#ids = list(set(cohort1.project_id))
#len(ids)

# Create a short list of ids for testing 
#ids = ['244eaf7e-e109-47b2-b84e-9ebe24508391', '24d8c9a2-b8ef-481c-930b-78c9aeaf239e', 'f17dd6cf-8187-4edd-895e-07013d4990c9', '1115dda6-0165-4099-b52f-0ac53595c3a9']
#len(ids)

In [ ]:
# Create a list of project IDs to query
## From the list of Batch 2 projects (8/26/25)
tree_count_proj_ids = list(batch2_projects['project_id'].unique())
tree_count_proj_ids

## Pull projects from TerraMatch API

In [ ]:
results = api.pull_wrapper(prod_url, headers, tree_count_proj_ids, outfile=tm_api_pull_results_file)

## Parse and save the API output

In [ ]:
# Load the saved JSON file
with open(tm_api_pull_results_file, 'r') as file:
    results = json.load(file)

In [ ]:
# Convert the JSON output into a dataframe with selected fields
results_df = api.parse_tm_api_results(results, outfile = polygon_features_file, parse_indicators=True)

### Filter PPC polygons by desired plantstart years

In [ ]:
# Read in polygons dataframe
df = pd.read_csv(polygon_features_file)

In [ ]:
print(df.shape)
df.head()

In [ ]:
# Filter the polygons dataframe by the year(s) of interest
results_df_filt = clean.filter_by_years_of_interest(polygons_df=df, years_df=batch2_projects)

In [ ]:
# Save the filtered dataframe (overwrite the unfiltered data)
results_df_filt.to_csv(f"../data/{run_dir}/tm_api_{run_name}_{today}.csv", index=False)

In [ ]:
print(f"results_df has {len(results_df)} polygons")
print(f"results_df_filt has {len(results_df_filt)} polygons")